# 6. Domknięcia i dekoratory

## 6.1. Zasięg zmiennych

In [2]:
gl = 101

def fun():
    print(gl)
    
fun()

101


Python nie jest w stanie modyfikować zmiennych spoza obecnego zakresu:

In [3]:
gl = 102

def fun():
    gl += 1
    print(gl)
    
fun()

UnboundLocalError: local variable 'gl' referenced before assignment

To zadziała:

In [4]:
gl = 102

def fun():
    global gl
    gl += 1
    print(gl)
    
fun()

103


Python stworzy lokalną zmienną o tej samej nazwie:

In [5]:
gl = 102

def fun():
    gl = 102
    gl += 1
    print(gl)
    
fun()

103


Python sam "rozgryza" zasięg zmiennych, jeśli mu nie podpowiemy

In [ ]:
gl = 105
check = 200

def outer():
    gl = 106
    def inner():
        return gl, check
    return inner()

outer()

Python ma cztery poziomy na których szuka zmiennych:
* lokalny (local)
* otaczającej funkcji (enclosing function)
* globalny (global)
* wbudowany (built-in)

## 6.2. Domknięcia (closures)

To nie jest domknięcie:

In [8]:
def fib(n):
    if n == 1 or n == 2:
        return 1
    a, b = 1, 1
    
    def recalculate(a, b):
        a, b = a + b, a
        return a, b
    
    for i in xrange(2, n):
        a, b = recalculate(a, b)
    return a

print fib(3)
print fib(4)

2
3
{}


Są to po prostu zagnieżdżone funkcje. Prostszy przykład:

In [11]:
import dis


def outer(a):
    
    def inner(b):
        return 2 * b

    return a + inner(a)


dis.dis(outer)

  6           0 LOAD_CONST               1 (<code object inner at 0x7f28780fc630, file "<ipython-input-11-c4d495c731e5>", line 6>)
              3 MAKE_FUNCTION            0
              6 STORE_FAST               1 (inner)

  9           9 LOAD_FAST                0 (a)
             12 LOAD_FAST                1 (inner)
             15 LOAD_FAST                0 (a)
             18 CALL_FUNCTION            1
             21 BINARY_ADD          
             22 RETURN_VALUE        


A to już jest domknięcie:

In [13]:
def outer(a):
    
    def inner():
        return a + 2*a
    
    return inner()

dis.dis(outer)

  3           0 LOAD_CLOSURE             0 (a)
              3 BUILD_TUPLE              1
              6 LOAD_CONST               1 (<code object inner at 0x7f28784596b0, file "<ipython-input-13-d95b43d920dc>", line 3>)
              9 MAKE_CLOSURE             0
             12 STORE_FAST               1 (inner)

  6          15 LOAD_FAST                1 (inner)
             18 CALL_FUNCTION            0
             21 RETURN_VALUE        


Domknięcie w pythonie jest specjalnym, wydzielonym konstruktem. W szczególności powyższy zapis raczej rzadko się przydaje. Coś bardziej przydatnego:

In [18]:
class db(object):
    @classmethod
    def conn(cls, *args, **kwargs):
        pass

def gen_db_connector(host):
    def make_db_connection(username, passwd):
        return db.conn(host=host, username=username, passwd=passwd)
    return make_db_connection
        
connector = gen_db_connector('10.0.0.10')
connector(username='Pudzian', passwd='Sila')

Domknięcie może realizować wzorzec fabryki. Inny przykład:

In [29]:
from functools import partial

def add(a, b):
    return a + b

add3 = partial(add, 3)
add3(3)

6

Funkcja wewnętrzna nie może modyfikować zmiennych z funkcji otaczającej (tak samo jak w przypadku zmiennych globalnych):

In [33]:
def outer(param):
    calc = param*3
    
    def inner():
        calc += 1
    return inner

outer(3)()

UnboundLocalError: local variable 'calc' referenced before assignment

Domknięcie:
* może być użyte do implemetacji wzorca fabryki
* używane jest do implementacji dekoratorów

## 6.3. Dekoratory

Dekorator to takie domknięcie, które jako argument przyjmuje inną funkcję.
W pythonie funkcje są obywatelami pierwszej kategorii (ang. first class citizens).

In [4]:
def decorator(func):
    
    def decorating():
        print "Decorating.."
        return func()
    return decorating

@decorator
def write3():
    print 3
    
        
write3()

Decorating..
3


Funkcja dekorująca podmienia oryginalną funkcję.

In [11]:
import time
import random

def timeit(func):
    
    def decorating():
        t1 = time.time()
        res = func()
        t2 = time.time()
        print("Elapsed: {}".format(t2 - t1))
        return res
    return decorating


def is_prime(n):
    i = 2
    while i * i <= n:
        if n % i == 0:
            return False
        i += 1
    return True


@timeit
def only_prime():
    start = random.randint(1, 40)
    print("Checking: {}".format(start))
    i = 0
    while not is_prime(start):
        start = start * 3 + 1
        if start > 10**20:
            return -1
        i += 1
    return start

only_prime()

Checking: 10
Elapsed: 9.29832458496e-05


31

A co z argumentami?

In [16]:
def timeit(func):
    
    def decorating(*args, **kwargs):
        t1 = time.time()
        res = func(*args, **kwargs)
        t2 = time.time()
        print("Elapsed: {}".format(t2 - t1))
        return res
    return decorating


@timeit
def only_prime(start):
    print("Checking: {}".format(start))
    i = 0
    while not is_prime(start):
        start = start * 3 + 1
        if start > 10**22:
            return -1
        i += 1
    return start


print only_prime(6)
print only_prime(35)

Checking: 6
Elapsed: 0.000513076782227
19
Checking: 35
Elapsed: 0.000939130783081
-1


Args i kwargs przypomnienie: 

In [17]:
def show_it(*args, **kwargs):
    print args
    print kwargs
    
show_it(16, "koktajl", zimno="cieplo")

(16, 'koktajl')
{'zimno': 'cieplo'}


Dekorator inaczej:

In [20]:
show_it_timeit = timeit(show_it)

show_it_timeit("zupa")

show_it("niezupa")

('zupa',)
{}
Elapsed: 0.000369071960449
('niezupa',)
{}


Jeden mały problem:

In [21]:
only_prime.__name__

'decorating'

Rozwiązanie:

In [24]:
from functools import wraps

def timeit(func):
    
    @wraps(func)
    def decorating(*args, **kwargs):
        t1 = time.time()
        res = func(*args, **kwargs)
        t2 = time.time()
        print("Elapsed: {}".format(t2 - t1))
        return res
    return decorating


@timeit
def make_my_day():
    print "Done"
    
make_my_day()
make_my_day.__name__

Done
Elapsed: 0.000396966934204


'make_my_day'

## 6.4. Dekoratory z argumentami

Napiszmy dekorator, który będzie powtarzał wykonanie funkcji (retry) tylko, gdy złapie określony błąd np. ConnectionError.

In [27]:
class ConnectionError(Exception):
    pass

RETRY_CNT = 5


def retry(func):
    
    def decorating(*args, **kwargs):
        i = 0
        while i < RETRY_CNT:
            try:
                res = func(*args, **kwargs)
                return res
            except ConnectionError:
                i += 1
                print("Reconnecting cnt: {}".format(i))
    return decorating

@retry
def do_not_even_try_to_connect():
    raise ConnectionError()

do_not_even_try_to_connect()

Reconnecting cnt: 1
Reconnecting cnt: 2
Reconnecting cnt: 3
Reconnecting cnt: 4
Reconnecting cnt: 5


Jak tu wcisnąć argument?

In [32]:
class HeisenbugException(Exception):
    pass


def retry(error_type):
    
    def decorator(func):
    
        @wraps(func)
        def decorating(*args, **kwargs):
            i = 0
            while i < RETRY_CNT:
                try:
                    res = func(*args, **kwargs)
                    return res
                except error_type:
                    i += 1
                    print("Reconnecting cnt: {} in func: {}".format(i, func.__name__))
        return decorating
    return decorator


@retry(ConnectionError)
def do_not_even_try_to_connect():
    raise ConnectionError()
    
@retry(ConnectionError)
def connect_but_fail_annoyingly():
    raise HeisenbugException()

do_not_even_try_to_connect()
connect_but_fail_annoyingly()

Reconnecting cnt: 1 in func: do_not_even_try_to_connect
Reconnecting cnt: 2 in func: do_not_even_try_to_connect
Reconnecting cnt: 3 in func: do_not_even_try_to_connect
Reconnecting cnt: 4 in func: do_not_even_try_to_connect
Reconnecting cnt: 5 in func: do_not_even_try_to_connect


HeisenbugException: 

## 6.5. Zastosowanie

Dekoratory zawierają wspólny, powtarzalny kod, który jest niezwiązany z właściwą logiką dekorowanych funkcji.

Przykłady:
* profilowanie
* powtarzanie wywołania
* zliczanie, ratelimiting
* logowanie
* oznaczanie właściwości kodu np. deprecated
* mockowanie
* kontrola wywołania funckji np. wymuszanie polityk (policies)

Kontrola wywołania w Openstack Heat:

In [34]:
def wrappertask(task):
    """
    Decorator for a task that needs to drive a subtask.

    This is essentially a replacement for the Python 3-only "yield from"
    keyword (PEP 380), using the "yield" keyword that is supported in
    Python 2. For example::

        @wrappertask
        def parent_task(self):
            self.setup()

            yield self.child_task()

            self.cleanup()
    """

    @functools.wraps(task)
    def wrapper(*args, **kwargs):
        parent = task(*args, **kwargs)

        subtask = next(parent)

        while True:
            try:
                if subtask is not None:
                    subtask_running = True
                    try:
                        step = next(subtask)
                    except StopIteration:
                        subtask_running = False

                    while subtask_running:
                        try:
                            yield step
                        except GeneratorExit as ex:
                            subtask.close()
                            raise ex
                        except:  # noqa
                            try:
                                step = subtask.throw(*sys.exc_info())
                            except StopIteration:
                                subtask_running = False
                        else:
                            try:
                                step = next(subtask)
                            except StopIteration:
                                subtask_running = False
                else:
                    yield
            except GeneratorExit as ex:
                parent.close()
                raise ex
            except:  # noqa
                subtask = parent.throw(*sys.exc_info())
            else:
                subtask = next(parent)

    return wrapper

Sprawdzanie dostępów:

In [ ]:
def policy_enforce(handler):
    '''
    Decorator for a handler method that checks the path matches the
    request context and enforce policy defined in policy.json
    '''
    @wraps(handler)
    def handle_stack_method(controller, req, tenant_id, **kwargs):
        if req.context.tenant_id != tenant_id:
            raise exc.HTTPForbidden()
        allowed = req.context.policy.enforce(context=req.context,
                                             action=handler.__name__,
                                             scope=controller.REQUEST_SCOPE)
        if not allowed:
            raise exc.HTTPForbidden()
        return handler(controller, req, **kwargs)

    return handle_stack_method

## ZADANIE

Napisz dekorator, który policzy wywołania funkcji read oraz write poniższej klasy. Można wykorzystać w tym celu metody wbudowane "getattr" i "setattr".
https://docs.python.org/2/library/functions.html#getattr

In [6]:
def count(f):
    pass


class Filesystem(object):
    
    def __init__(self):
        self.read_cnt = 0
        self.write_cnt = 0

    @count
    def read(self, filename):
        print "Read {}".format(filename)
    
    @count
    def write(self, filename, text):
        print "Wrote {}".format(text)

f = Filesystem()
f.read("fst_file")
f.read("sec_file")
f.write("thr_file", "nobody expects the spanish inquisition")
print f.read_cnt
print f.write_cnt

TypeError: 'NoneType' object is not callable